To be up-to-date on the most current version of this code. Check out our GitHub repository: https://github.com/Neatherblok/SnowDetection

In [1]:
#!pip install datasets
#!pip install --upgrade --force-reinstall huggingface_hub

In [ ]:
import torch
import torchvision
from torchvision.transforms import transforms, Lambda, Resize
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import os

## Data Preparation

In [4]:

'''
Creates a class object CustomDataLoader
************
__init__
  This function initializes a CustomDataLoader object.
  It loads a part of the ImageNet-1K dataset specified by the user.
  Options for dataset_type are 'train' and 'test'.

  Args:
    data_path (str): Path to the dataset.
    batch_size (int): Batch size for data loading.
    dataset_type (str): Specifies whether the dataset is for training or testing.

************
__getitem__
  This function returns two variables containing each a variable present 
  in the original ImageNet-1K dataset.
  Before returning these variables, it splits up and transforms the image 
  into a fixed resolution of 48 by 48 pixels.
  
  Returns:
    data: The transformed image data.
    label: The label corresponding to the image.
************
__len__
  Returns: the length of the loaded dataset.
'''

class CustomDataLoader:
    def __init__(self, data_path, batch_size, dataset_type, mean=None, std=None):
        self.data_path = data_path
        self.batch_size = batch_size
        self.dataset_type = dataset_type
        self.mean = mean
        self.std = std

        if mean is None or std is None:
            self.transform = transforms.Compose([
                transforms.Resize(128),  # Resize to size 128x128,
                transforms.ToTensor()
            ])

        else: 
            self.transform = transforms.Compose([
                transforms.Resize(128),  # Resize to size 128x128,
                transforms.ToTensor(),
                transforms.Normalize(mean=self.mean, std=self.std)
            ])

        self.dataset = torchvision.datasets.ImageFolder(root=f"{self.data_path}/{dataset_type}",
                                                        transform=self.transform)

        if self.dataset_type == 'train':
            shuffle = True
        else:
            shuffle = False

        self.data_loader = DataLoader(self.dataset, batch_size=self.batch_size, shuffle=shuffle,
                                      num_workers=os.cpu_count(),
                                      pin_memory=True if torch.cuda.is_available() else False)

    def __getitem__(self, index):
        return self.dataset[index]

    def __len__(self):
        return len(self.dataset)

In [5]:
# Specify image dataset path:
DATA_PATH = "../../data"

# Specify image batch size
BATCH_SIZE = 4

# Load and execute transformations on datasets
train_data_loader = CustomDataLoader(DATA_PATH, BATCH_SIZE, dataset_type='train').data_loader
test_data_loader = CustomDataLoader(DATA_PATH, BATCH_SIZE, dataset_type='test').data_loader

In [6]:
print("Train Data Loader:")
for batch_idx, (inputs, labels) in enumerate(train_data_loader):
    print("Batch Index:", batch_idx)
    print("Inputs Shape:", inputs.shape)
    print("Labels Shape:", labels.shape)
    # Print the first few labels in the batch
    print("Labels:", labels[:5])
    # Break after printing a few batches
    if batch_idx == 2:
        break

Train Data Loader:
Batch Index: 0
Inputs Shape: torch.Size([4, 3, 128, 128])
Labels Shape: torch.Size([4])
Labels: tensor([0, 0, 1, 1])
Batch Index: 1
Inputs Shape: torch.Size([4, 3, 128, 128])
Labels Shape: torch.Size([4])
Labels: tensor([1, 0, 0, 1])
Batch Index: 2
Inputs Shape: torch.Size([4, 3, 128, 128])
Labels Shape: torch.Size([4])
Labels: tensor([1, 0, 0, 0])


In [7]:
# Get the class labels associated with the dataset
class_labels = train_data_loader.dataset.classes

# Print out the class labels
print("Class Labels:", class_labels)

Class Labels: ['no-snow', 'snow']


In [8]:
# Calculate the total number of images in the train data loader
total_train_images = len(train_data_loader.dataset)

# Print out the total number of images
print("Total number of images in train data loader:", total_train_images)

Total number of images in train data loader: 78
